# 6 - Run Optimising Modulation: Reducing the Network Connections (Generates Data for Figure 14) 
 

This notebook re-generates data for the modulation tests (figure 14) in *Section 9.3: Optimising Sub-pattern Recognition*. 

**To just plot the results:** 
If you would like to just plot the pre-generated paper results, there is no need to re-generate the results as they are cached in `examples/experimental_results/experiment_B_modulation_e` and `examples/experimental_results/experiment_B_modulation_i`. In this case, simply use the Jupyter notebook `7 - Experiment_B_modulation - Plot Fig 14.ipynb`. 

**Where the data will be saved:** The results from this test willl be saved in the existing Excel files in the `examples/experimental_results/experiment_B_modulation_e` and `examples/experimental_results/experiment_B_modulation_i` folders and overwrite the sheet named `latest_data`. The data generated for the paper is on the `paper_data` sheet, which will not be overwritten. You can select the data to plot (the latest or the paper data) by selecting the correct sheet name in `7 - Experiment_B_modulation - Plot Fig 14.ipynb`.

This notebook explores the effect on the modulation step through one optimisation technique: 

* Optimising the feature to hidden neuron connections

The purpose of this test is to establish whether this optimisation can reduce the number of modulation connections required, while maintaining accuracy.

To examine the effect of this optimisation on ESAM network robustness, the following experiment is used:

<table>
<tr><td>

| Problem B    |          |
| :----------- | :------  | 
| $f$          | 2,000      | 
| $m$          | 1,000 | 
| $s_m$        | 0.1     | 
| $s_n$        | Varying      | 
</td><td>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
</td><td>

| Network B (Baseline) |          |
| :--------------------| :------  | 
| $h$                  | 2        | 
| $\theta$             | 12       | 
| $s^+_{f\_h}$         | 0.1      | 
| $\sigma_{h\_pre+}$   | `FixedProbability` | 
| $s^+_{h\_f}$         | 0-1 (set to 1 when  $s^-_{h\_f}$ is varied)   | 
| $s^-_{h\_f}$         | 0-1 (set to 1 when  $s^+_{h\_f}$ is varied)   | 
| $\sigma_{f\_pre+}$, $\sigma_{f\_pre-}$  |  `FixedProbability`       | 
| $e$                  | 3        | 

</td></tr> </table>

This is based on the baseline experiment presented in [1] (figure 6).

The accuracy of two different networks are compared in the graphs as the excitatory $s^+_{h\_f}$ and inhibitory $s^-_{h\_f}$ modulation sparsities vary:

* Network B - the baseline network (as described above).
* Network B with the modulation connections optimised as described in section 7.1.


[1] Hoffmann, H. (2019). Sparse Associative Memory. *Neural Computation*

## Dependencies and problem/network definition

Import the dependencies and define the tests to run. 

### Dependencies

In [1]:
import os
import sys
sys.path.append('../src')
import numpy as np

In [2]:
output_base_dir = '.'+os.sep+'experiment_results'

### Define the problem space and base network

Provide the default values for the problem space and the network. The tests will be variations 
on these defaults.

In [3]:
# Default values for the problem and network
f = 2000
s_m = 200/f

problem_space_default = {'f': f,
                         'm': 1000,
                         's_m': s_m,
                         's_n': 0.1,
                         's_is_prob': False}

network_default = {'f': f,
                   'h': 2,
                   'f_h_sparsity': 0.1,
                   'h_f_sparsity_e': 1,
                   'h_f_sparsity_i': 1,
                   'e': 3,
                   'h_thresh': 12,
                   'f_h_conn_type': 'FixedProbability',
                   'h_f_conn_type': 'FixedProbability',
                   'debug': False
                   }


### Define the variable that will change (the x-axis on the final plots)

In this case, the variable that is changed is the hidden to feature neuron connection sparsity `h_f_sparsity_e`.

In [4]:
# Below is the full test.
vary_sparsity = np.arange(0, 1.1, 0.1)

# The full test will take a long time to run. You can experiment with fewer memories.
# You can also reduce the number of simulations that are run per memory variation - see the 
# 'Run the Tests' section below. 
vary_sparsity = np.arange(0, 1.1, 0.5)

### Define the networks to be tested. 

Each network will be tested multiple times for each of the variable options.

The `problem_static` and `network_vary` dictionary keys indicate that the variation betweeen the tests is the network - specifically in this case, the hidden to feature neuron excitatory connectivity `h_f_sparsity_e`.

There are 2 network varieties:
* `original`: the original network from the paper.
* `opt_conns`: the original network with the hidden to feature (modulation) connections optimised.

In [5]:
import copy

original={
            'problem_static': problem_space_default.copy(),
            'network_vary': network_default.copy(),
            'plot_params':
                {'variable_column': 'h_f_sparsity',
                 'variable_column_name': 'Modulation connections',
                 'variable_is_integer': False}
         }

# This is the optimisated network with a static number of pre-synaptic connections to each feature neuron. 
opt_conns = copy.deepcopy(original)
opt_conns['network_vary']['h_f_conn_type'] = 'FixedNumberPre'

Put all the network descriptions into a single dictionary for the test runner.

In [6]:
all_tests = {'original': original,
             'opt_conns': opt_conns
            }

## Run the tests

`num_tests` is the number of networks for each type of test that will be created. Despite having the same network definition, each will vary due to randomness in the data and network connections.

`simulations_per_test` is the number of recalls that will be run on each test network.

The total number of tests that will be run is: `number of test networks * len(network_vary) * num_tests * simulations_per_test`

This test is classified as `vary_network` because the problem space remains static for each optimisation test, but the network is varied.

In [7]:
num_tests = 10
sims_per_test = 10

## Run Tests with Varying Excitatory Connections

In [8]:
test_name = 'experiment_B_modulation_e'
output_dir = output_base_dir + os.sep + test_name

# Override with the variable parameter. Specifying an array for this parameter will
# indicate to the test runner that there are multiple scenarios to be tested.
original['network_vary']['h_f_sparsity_e'] = vary_sparsity
opt_conns['network_vary']['h_f_sparsity_e'] = vary_sparsity
original['network_vary']['h_f_sparsity_i'] = 1
opt_conns['network_vary']['h_f_sparsity_i'] = 1

In [9]:
from simulation_scripts.test_suite_runner import TestSuiteRunner

tsr = TestSuiteRunner(network_default=network_default,
                      problem_space_default=problem_space_default,
                      output_dir=output_dir)

tsr.run_vary_network_test_suite(all_tests = all_tests,
                                num_tests = num_tests,
                                sims_per_test = sims_per_test)


Running test set: original defined as: 
    Static problem space:    {'f': 2000, 'm': 1000, 's_m': 0.1, 's_n': 0.1, 's_is_prob': False}
    Varying network params:  {'f': 2000, 'h': 2, 'f_h_sparsity': 0.1, 'h_f_sparsity_e': array([0. , 0.5, 1. ]), 'h_f_sparsity_i': 1, 'e': 3, 'h_thresh': 12, 'f_h_conn_type': 'FixedProbability', 'h_f_conn_type': 'FixedProbability', 'debug': False}

Number of tests per scenario: 10 
Number of simulations per test: 10 
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               200
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for no

Recall complete. Average time per test (ms) was:  36.78
  Simulation complete
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for sign

Recall complete. Average time per test (ms) was:  36.85
  Simulation complete
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for sign

Recall complete. Average time per test (ms) was:  36.79
  Simulation complete
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for sign

INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: More than closest memory was found for signal. 
      Closest [41] was selected from [ 8 41]. All had the signal similarity measure: [0.095]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the si

...

INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measu

1000 memories learnt in 2.1s. Approx time per memory: 2.1ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  41.69
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [0] was selected from [  0 711]. All had the signal similarity measure: [0.13]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: More than closest memory was found for signal. 
      Closest [158] was selected from [  3 158]. All had the signal similarity measure: [0.15]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: More than clos

1000 memories learnt in 2.05s. Approx time per memory: 2.05ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  39.45
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [431] was selected from [  2 431]. All had the signal similarity measure: [0.015]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: More than closest memory was found for signal. 
      Closest [794] was selected from [  4 794]. All had the signal similarity measure: [0.125]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No close

1000 memories learnt in 2.04s. Approx time per memory: 2.04ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  38.91
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [0] was selected from [  0 102]. All had the signal similarity measure: [0.1]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: More than closest memory was found for signal. 
      Closest [911] was selected from [  3 911]. All had the signal similarity measure: [0.14]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest me

1000 memories learnt in 2.04s. Approx time per memory: 2.04ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  39.09
  Simulation complete
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0

Results (final epoch) analysis using ground truth memories:
     Proportion of results correct for each epoch:  [0.  0.9 0.9 0.9]
  Learning phase...

=========== ESAM Recall Network =============

    Num features:                   2000
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1.0
    H to F inhibitory sparsity:     1
      Static H to F conn type?      FixedNumberPre
    Max epochs per recall:          3
    Hidden per memory:              2
    Hidden threshold:       

In [10]:
print('Experiment complete, data stored in:\n')
print('  {:80s}    {:10s}\n'.format('File', 'Sheet'))

for test_file, _ in all_tests.items():
    excel_file = output_base_dir + os.sep + test_name + os.sep + test_file + '.xlsx'
    print('  {:80s}    {:10s}'.format(excel_file, 'latest'))

Experiment complete, data stored in:

  File                                                                                Sheet     

  ./experiment_results/experiment_B_modulation_e/original.xlsx                        latest    
  ./experiment_results/experiment_B_modulation_e/opt_conns.xlsx                       latest    


## Run Tests with Varying Inhibitory Connections

In [11]:
test_name = 'experiment_B_modulation_i'
output_dir = output_base_dir + os.sep + test_name

# Override with the variable parameter. Specifying an array for this parameter will
# indicate to the test runner that there are multiple scenarios to be tested.
original['network_vary']['h_f_sparsity_e'] = 1
opt_conns['network_vary']['h_f_sparsity_e'] = 1
original['network_vary']['h_f_sparsity_i'] = vary_sparsity
opt_conns['network_vary']['h_f_sparsity_i'] = vary_sparsity

In [12]:
print(original)

{'problem_static': {'f': np.int64(2000), 'm': 1000, 's_m': 0.1, 's_n': 0.1, 's_is_prob': False}, 'network_vary': {'f': array([2000, 2000, 2000]), 'h': array([2, 2, 2]), 'f_h_sparsity': array([0.1, 0.1, 0.1]), 'h_f_sparsity_e': 1, 'h_f_sparsity_i': array([0. , 0.5, 1. ]), 'e': array([3, 3, 3]), 'h_thresh': array([12, 12, 12]), 'f_h_conn_type': array(['FixedProbability', 'FixedProbability', 'FixedProbability'],
      dtype='<U16'), 'h_f_conn_type': array(['FixedProbability', 'FixedProbability', 'FixedProbability'],
      dtype='<U16'), 'debug': array([False, False, False]), 'cols': array([1., 1., 1.])}, 'plot_params': {'variable_column': 'h_f_sparsity', 'variable_column_name': 'Modulation connections', 'variable_is_integer': False}}


In [13]:
from simulation_scripts.test_suite_runner import TestSuiteRunner

tsr = TestSuiteRunner(network_default=network_default,
                      problem_space_default=problem_space_default,
                      output_dir=output_dir)


tsr.run_vary_network_test_suite(all_tests = all_tests,
                                num_tests = num_tests,
                                sims_per_test = sims_per_test)


Running test set: original defined as: 
    Static problem space:    {'f': np.int64(2000), 'm': 1000, 's_m': 0.1, 's_n': 0.1, 's_is_prob': False}
    Varying network params:  {'f': array([2000, 2000, 2000]), 'h': array([2, 2, 2]), 'f_h_sparsity': array([0.1, 0.1, 0.1]), 'h_f_sparsity_e': 1, 'h_f_sparsity_i': array([0. , 0.5, 1. ]), 'e': array([3, 3, 3]), 'h_thresh': array([12, 12, 12]), 'f_h_conn_type': array(['FixedProbability', 'FixedProbability', 'FixedProbability'],
      dtype='<U16'), 'h_f_conn_type': array(['FixedProbability', 'FixedProbability', 'FixedProbability'],
      dtype='<U16'), 'debug': array([False, False, False]), 'cols': array([1., 1., 1.])}

Number of tests per scenario: 10 
Number of simulations per test: 10 
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for

Recall complete. Average time per test (ms) was:  38.45
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [3] was selected from [  3 991]. All had the signal similarity measure: [0.54054054]
INFO: More than closest memory was found for signal. 
      Closest [3] was selected from [  3 383 991]. All had the signal similarity measure: [0.37593985]
INFO: More than closest memory was found for signal. 
      Closest [495] was selected from [  3 152 251 309 383 495 571 747 786 820 991]. All had the signal similarity measure: [0.14716703]
INFO: More than closest memory was found for signal. 
      Closest [4] was selected from [  4  17 216]. All had the signal similarity measure: [0.37664783]
INFO: More than closest memory was found for signal. 
      Closest [340] was selected from [  4  17  68 152 195 216 286 340 383 435 469 597 710 757 797 828 840 844
 878]. All had the signal similarity measure: [0.11695906]
INFO: No closest memories found for sign


Results (final epoch) analysis using ground truth memories:
     Proportion of results correct for each epoch:  [0.  0.9 0.9 0.9]
  Learning phase...

=========== ESAM Recall Network =============

    Num features:                   2000
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1
    H to F inhibitory sparsity:     0.0
      Static H to F conn type?      FixedProbability
    Max epochs per recall:          3
    Hidden per memory:              2
    Hidden threshold:               12
    Debug on:                       False

>> Learning 1000 memories
1000 memories learnt in 0.08s. Approx time per memory: 0.08ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  38.14
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [0] was selected from [  0 918]. All had the signal similarity measure: [0.53908356]
INFO: More than c

Recall complete. Average time per test (ms) was:  38.28
  Simulation complete

Results (final epoch) analysis using ground truth memories:
     Proportion of results correct for each epoch:  [0.  0.4 0.7 0.8]
  Learning phase...

=========== ESAM Recall Network =============

    Num features:                   2000
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1
    H to F inhibitory sparsity:     0.5
      Static H to F conn type?      FixedProbability
    Max epochs per recall:          3
    Hidden per memory:              2
    Hidden threshold:               12
    Debug on:                       False

>> Learning 1000 memories
1000 memories learnt in 0.1s. Approx time per memory: 0.1ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  38.03
  Simulation complete

Results (final epoch) analysis using ground truth memories:
     Proportion of results correc

Recall complete. Average time per test (ms) was:  37.91
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [346] was selected from [  1 346]. All had the signal similarity measure: [0.125]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.0

Results (final epoch) analysis using ground truth memories:
     Proportion of results correct for each epoch:  [0.  0.9 0.9 0.9]
  Learning phase...

=========== ESAM Recall Network =============

    Num features:                   2000
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1
    H to F inhibitory sparsity:     1.0
      Static H to F conn type?      FixedProbability
    Max epochs per recall:          3
    Hidden per mem

======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               200
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               200
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               200
======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      N


Results (final epoch) analysis using ground truth memories:
     Proportion of results correct for each epoch:  [0.  0.7 0.7 0.7]
  Learning phase...

=========== ESAM Recall Network =============

    Num features:                   2000
    F to H connection sparsity:     0.1
      Static F to H conn type?      FixedProbability
    H to F exitatory sparsity:      1
    H to F inhibitory sparsity:     0.0
      Static H to F conn type?      FixedNumberPre
    Max epochs per recall:          3
    Hidden per memory:              2
    Hidden threshold:               12
    Debug on:                       False

>> Learning 1000 memories
1000 memories learnt in 0.25s. Approx time per memory: 0.25ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  38.05
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [100] was selected from [  2 100]. All had the signal similarity measure: [0.53475936]
INFO: More than c

1000 memories learnt in 0.25s. Approx time per memory: 0.25ms
  Recall phase...

>> Recalling 10 tests.
Recall complete. Average time per test (ms) was:  38.18
  Simulation complete
INFO: More than closest memory was found for signal. 
      Closest [906] was selected from [  4  93 448 906 949]. All had the signal similarity measure: [0.24449878]
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.1
INFO: No closest memories found for signal. 
      Closest was set to -1. All memories had the signal similarity measure: 0.1
INFO: More than closest memory was found for signal. 
      Closest [6] was selected from [  6 136]. All had the signal similarity measure: [0.53191489]
INFO: More than closest memory was found for signal. 
      Closest [136] was selected from [  6 136 448]. All had the signal similarity measure: [0.37383178]
INFO: More than closest memory was found for signal. 
      Closest [448] was selected 

INFO: More than closest memory was found for signal. 
      Closest [204] was selected from [  3 204]. All had the signal similarity measure: [0.53333333]
INFO: More than closest memory was found for signal. 
      Closest [204] was selected from [  3 204 716]. All had the signal similarity measure: [0.37453184]
INFO: More than closest memory was found for signal. 
      Closest [228] was selected from [  3  19  49 104 136 179 195 204 228 389 716 721 811 964 965]. All had the signal similarity measure: [0.12804097]
INFO: More than closest memory was found for signal. 
      Closest [8] was selected from [  8 855]. All had the signal similarity measure: [0.51679587]
INFO: More than closest memory was found for signal. 
      Closest [220] was selected from [  8 220 855]. All had the signal similarity measure: [0.36429872]
INFO: More than closest memory was found for signal. 
      Closest [43] was selected from [  5   8  21  43 220 228 267 358 443 460 636 658 853 855 859 988 992]. All h

In [14]:
print('Experiment complete, data stored in:\n')
print('  {:80s}    {:10s}\n'.format('File', 'Sheet'))

for test_file, _ in all_tests.items():
    excel_file = output_base_dir + os.sep + test_name + os.sep + test_file + '.xlsx'
    print('  {:80s}    {:10s}'.format(excel_file, 'latest'))

Experiment complete, data stored in:

  File                                                                                Sheet     

  ./experiment_results/experiment_B_modulation_i/original.xlsx                        latest    
  ./experiment_results/experiment_B_modulation_i/opt_conns.xlsx                       latest    


## Next steps: Plotting the results

Use the Jupyter notebook `7 - Experiment_B_modulation - Plot Fig 14.ipynb` to plot the results.

You will need to update the `data_sheet` variable to `latest_data` as described in the notebook, else the pre-generated paper data will be plotted.